In [145]:
import string
import re
from unicodedata import normalize
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Bidirectional,GRU,LSTM,Dense,Embedding,RepeatVector,TimeDistributed,Activation,Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras import activations
import pandas as pd
import tensorflow as tf
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
import random
from random import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from random import Random
from numpy.random import seed
import time


In [146]:

df = pd.read_json('train.json')

final_test = pd.read_json('test_wor.json')

In [147]:
!pip install transformers

In [148]:
from transformers import AutoTokenizer, AutoModel

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("racai/distilbert-base-romanian-uncased")
model = AutoModel.from_pretrained("racai/distilbert-base-romanian-uncased")



Some weights of the model checkpoint at racai/distilbert-base-romanian-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [149]:
import tqdm
import numpy as np

def tokenize(sentences, tokenizer):
    input_ids= []
    for sentence in sentences:
        inputs = tokenizer.encode(sentence, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
        input_ids.append(inputs)       
        
    return np.asarray(input_ids, dtype='int32')

In [150]:
trainX = df["text"]
for idx, x in enumerate(trainX):
  trainX[idx] = x.lower()
r1 = tokenize(trainX, tokenizer)

final_testX = final_test["text"]
for idx, x in enumerate(final_testX):
  final_testX[idx] = x.lower()
final_r1 = tokenize(final_testX, tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [151]:
X = r1
Y = df["rating"]

dataX = []
dataY = []
count5 = 0
for x, y in zip(X, Y):
  if y == 5:
    count5 += 1

  # if y == 5 and count5 <= 3200:
  #   dataX.append(x)
  #   dataY.append(y)
  
  # if y!= 5:
  #   dataX.append(x)
  #   dataY.append(y)
  
  dataX.append(x)
  dataY.append(y)
  

test = dict(Counter(dataY))
test[0] = 0



final_X = final_r1

In [152]:
dataX = np.array(dataX)
dataY = np.array(dataY)
final_X = np.array(final_X)
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.33, random_state=42)


new_X_train = []
new_Y_train = []
count5=0
for x, y in zip(X_train, y_train):
  if y == [5]:
    count5 += 1

  # if y == 5 and count5 <= 4000:
  #   new_X_train.append(x)
  #   new_Y_train.append(y)
  
  # if y!= 5:
  #   new_X_train.append(x)
  #   new_Y_train.append(y)
  new_X_train.append(x)
  new_Y_train.append(y)


X_train = np.array(new_X_train)
y_train = np.array(new_Y_train)


print(dict(Counter(y_train)))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


{5: 11081, 1: 1220, 4: 2173, 3: 1153, 2: 483}


In [153]:
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [154]:
from keras import backend as K



In [155]:
def get_f1(y_true, y_pred): 
  return metrics.f1_score(y_true, y_pred, average='weighted')

In [156]:
# X_train = np.array(X_train)

In [157]:
# X_train = np.array(X_train.tolist())

In [158]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
def plot_history(history):
  # print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [159]:
{5: 11081, 1: 1220, 4: 2173, 3: 1153, 2: 483}

{1: 1220, 2: 483, 3: 1153, 4: 2173, 5: 11081}

In [160]:
test_new ={
    0:1,
    1:5,
    2:10,
    3:5,
    4:3,
    5:1
}

In [161]:
from sklearn import metrics
import numpy as np
import json
global max_f1
max_f1 = 0
def predict_store_data(model, modelname):
  predictions = model.predict(X_test)

  y_pred = []
  for x in predictions:
    y_pred.append(np.argmax(x))

  y_test2 = []
  for x in y_test:
    y_test2.append(np.argmax(x))

  f1_weighted = get_f1(y_test2, y_pred)


  final_predictions = model.predict(final_X)
  y_final = []
  for x in final_predictions:
    y_final.append(np.argmax(x))

  for idx, y in enumerate(y_final):
    final_test["rating"][idx] = y

  with open('/content/drive/MyDrive/SII/results.txt', 'a') as f:
          f.write("%s; %s;\n" % (modelname, f1_weighted))

  global max_f1
  if(f1_weighted > max_f1):
    max_f1 = f1_weighted
    final_test.to_json(path_or_buf="/content/drive/MyDrive/SII/"+ modelname+".json" , orient="records")
  
  return f1_weighted


In [166]:
#Defining Neural Network
def createModel(params):
  model = tf.keras.Sequential()
  model.add(Embedding(50000, params["emb"], input_length=512, mask_zero=True))
  model.add(Bidirectional(LSTM(params["lstm"]), merge_mode='concat'))
  model.add(Dropout(params["drop"]))
  model.add(tf.keras.layers.Dense(params["dense"]))
  model.add(Dropout(params["drop"]))
  model.add(Dense(6, activation="softmax"))

  optimizer = keras.optimizers.Adam(learning_rate=params["lr"])
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# model = createModel()
# # print(model.summary())
# history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1,
#         verbose=1,
#         # class_weight = test,
#         class_weight = test_new,
#         callbacks=[
#                       EarlyStopping(
#                       monitor='val_accuracy',
#                       patience=5,
#                       restore_best_weights=True
#                   )
#           ])

# plot_history(history)

lrs = [2e-4]
embs = [128]
drops = [0.8]
lstms = [10]
denses = [16]
model = None
for lr in lrs:
  for emb in embs:
    for drop in drops:
      for lstm in lstms:
        for dense in denses:
          seed(42)
          tf.random.set_seed(42)
          random.seed(42)

          params = {
              "lr":lr,
              "emb":emb,
              "drop":drop,
              "lstm":lstm,
              "dense":dense
          }

          model = createModel(params)

          history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1,
          verbose=1,
          class_weight = test_new,
          callbacks=[
                        EarlyStopping(
                        monitor='val_loss',
                        patience=4,
                        restore_best_weights=True
                    )
            ])
          
          f1_score = predict_store_data(model, "BILSTM2_test;" + json.dumps(params))
          print(f1_score, json.dumps(params))


Epoch 1/100
454/454 [==============================] - 24s 37ms/step - loss: 3.7122 - accuracy: 0.4783 - val_loss: 1.5422 - val_accuracy: 0.6853
Epoch 2/100
454/454 [==============================] - 15s 32ms/step - loss: 3.5872 - accuracy: 0.5765 - val_loss: 1.4157 - val_accuracy: 0.6853
Epoch 3/100
454/454 [==============================] - 15s 32ms/step - loss: 3.4552 - accuracy: 0.5916 - val_loss: 1.2724 - val_accuracy: 0.6983
Epoch 4/100
454/454 [==============================] - 15s 32ms/step - loss: 3.3443 - accuracy: 0.6050 - val_loss: 1.1488 - val_accuracy: 0.7039
Epoch 5/100
454/454 [==============================] - 16s 35ms/step - loss: 3.2276 - accuracy: 0.6201 - val_loss: 1.0635 - val_accuracy: 0.7027
Epoch 6/100
454/454 [==============================] - 15s 33ms/step - loss: 3.1388 - accuracy: 0.6402 - val_loss: 1.1390 - val_accuracy: 0.6685
Epoch 7/100
454/454 [==============================] - 15s 33ms/step - loss: 3.0543 - accuracy: 0.6538 - val_loss: 0.9276 - val_ac

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.6475676061480602 {"lr": 0.0002, "emb": 128, "drop": 0.8, "lstm": 10, "dense": 16}


In [163]:
test_input = tokenizer.encode("Microfon de calitate superioara. Izoralea fonica a castilor este foarte buna.", add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
predictions = model.predict([test_input])
print(np.argmax(predictions[0]))

4
